# CYK Algorithm for PCFG

In [2]:
from nltk import PCFG, ViterbiParser, InsideChartParser
import numpy as np
import pandas as pd

In [3]:
grammar = PCFG.fromstring("""
S -> NP VP [1.0]
VP -> V NP [0.7] | VP PP [0.3]
NP -> Det N [0.5] | NP PP [0.2] | 'John' [0.3]
PP -> P NP [1.0]
V -> 'saw' [0.2] | 'ate' [0.8]
Det -> 'the' [0.6] | 'a' [0.4]
N -> 'man' [0.5] | 'telescope' [0.5]
P -> 'with' [0.4] | 'in' [0.6]
""")

In [4]:
pcfg = PCFG.fromstring("""
S -> NP VP [0.9]
S -> VP [0.1]
VP -> V NP [0.5]
VP -> V [0.5]
NP -> Det N [0.3]
NP -> N [0.7]
N -> 'cat' [0.2]
N -> 'book' [0.2]
N -> 'bird' [0.2]
N -> 'dog' [0.4]
V -> 'read' [0.1]
V -> 'chased' [0.6]
V -> 'ate' [0.3]
Det -> 'the' [0.5]
Det -> 'a' [0.5]
""")
pcfg.productions()[1].rhs()

(VP,)

In [5]:
def cyk(pcfg, s):
    n = len(s)
    table = [[ [] for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for prod in pcfg.productions(rhs=s[i]):
            table[i][i] = [(prod.lhs(), prod.prob())]
    for l in range(2, n+1):
        for i in range(n-l+1):
            j=i+l-1
            for k in range(i,j):
                for prod in pcfg.productions():
                    for left, left_p in table[i][k]:
                        for right, right_p in table[k+1][j]:
                            if prod.rhs() == (left, right):
                                prob = left_p * right_p * prod.prob()
                                table[i][j].append((prod.lhs(),prob))
                                    

    if pcfg.start() in [lhs for lhs, prob in table[0][n-1]]:
        for lhs, prob in table[0][n-1]:
            if lhs == pcfg.start():
                return prob
    return 0.0

In [6]:
sent = [
"the cat chased the dog",
"the dog chased the cat",
"the dog chased the bird",
"Book read"
]

In [7]:
for s in sent:
    prob = cyk(pcfg, s.lower().split())
    if prob>0:
        print(f"The sentence {s} is grammatically correct. Prob = {prob}")
    else:
        print(f"The sentence {s} is grammatically incorrect")

The sentence the cat chased the dog is grammatically correct. Prob = 0.00048599999999999994
The sentence the dog chased the cat is grammatically correct. Prob = 0.00048599999999999994
The sentence the dog chased the bird is grammatically correct. Prob = 0.00048599999999999994
The sentence Book read is grammatically incorrect


In [10]:
print('''PARSE TREES FOR SENTENCES BELONGING TO THE GRAMMAR''')
parser = InsideChartParser(pcfg)
for s in sent:
    if cyk(pcfg,s.lower().split())>0:
        print(f"Probability of '{s}':{cyk(pcfg,s.lower().split())}")
        for tree in parser.parse(s.split()):
            tree.pretty_print()
    else:
        print(f"'{s}': grammatically incorrect")

PARSE TREES FOR SENTENCES BELONGING TO THE GRAMMAR
Probability of 'the cat chased the dog':0.00048599999999999994
              S               
      ________|_____           
     |              VP        
     |         _____|___       
     NP       |         NP    
  ___|___     |      ___|___   
Det      N    V    Det      N 
 |       |    |     |       |  
the     cat chased the     dog

Probability of 'the dog chased the cat':0.00048599999999999994
              S               
      ________|_____           
     |              VP        
     |         _____|___       
     NP       |         NP    
  ___|___     |      ___|___   
Det      N    V    Det      N 
 |       |    |     |       |  
the     dog chased the     cat

Probability of 'the dog chased the bird':0.00048599999999999994
              S                
      ________|_____            
     |              VP         
     |         _____|___        
     NP       |         NP     
  ___|___     |      ___|___ 